## Title Available On Request:

### An Introduction to Lazy Evaluation 

## Eager Evaluation

In [1]:
def f(a, b):
    """An arbitrary binary function.
    """
    return a + b

In [2]:
a = f(1, 2)

In [3]:
a

3

In [4]:
b = f(a, 3)

In [5]:
b

6

## Why Eager by Default?

## Lazy Evaluation

### Pure Functions

### Thunks

In [6]:
def f(a, b):
    """An arbitrary binary function.
    """
    return a + b

In [7]:
a = lambda: f(1, 2)

In [8]:
a

<function __main__.<lambda>>

In [9]:
b = lambda: f(a(), 3)

In [10]:
b

<function __main__.<lambda>>

In [11]:
b()

6

## Computations as Expression Trees

![](graphs/tree-full.svg)

![](graphs/tree-computed.svg)

## Extending to Directed Graphs

In [12]:
a = 1
b = 2
sub_expression = lambda: a + b

In [13]:
value = lambda: sub_expression() + sub_expression()

In [14]:
value()

6

![](graphs/redundant-tree.svg)

![](graphs/dag.svg)

## Cell Model Thunks

In [15]:
class CellThunk:
    not_evaluated = object()

    def __init__(self, code, *args, **kwargs):
        self.code = code
        self.args = args
        self.kwargs = kwargs
        self.value = self.not_evaluated
        
    def __call__(self):
        if self.value is self.not_evaluated:
            self.value = self.code(
                *(arg() for arg in self.args),
                **{key: value() for key, value in self.kwargs.items()},
            )
            del self.code
            del self.args
            del self.kwargs

        return self.value

In [16]:
a = CellThunk(lambda: 1)
b = CellThunk(lambda: 2)

In [17]:
def f(a, b):
    print(f'adding {a} and {b}')
    return a + b

sub_expression = CellThunk(f, a, b)

In [18]:
value = CellThunk(f, sub_expression, sub_expression)

In [19]:
value()

adding 1 and 2
adding 3 and 3


6

In [20]:
value()

6

## Self Updating Thunks

In [21]:
class SelfUpdatingThunk:
    def __init__(self, code, *args, **kwargs):
        def update_frame(*args, **kwargs):
            value = code(*args, **kwargs)
            
            self.code = self.indirection_code
            self.args = (lambda: value,)
            self.kwargs = {}
            
            return value
            
        self.code = update_frame
        self.args = args
        self.kwargs = kwargs
        
    @staticmethod
    def indirection_code(value):
        return value
    
    def __call__(self):
        return self.code(
            *(arg() for arg in self.args),
            **{key: value() for key, value in self.kwargs.items()},
        )

In [22]:
a = SelfUpdatingThunk(lambda: 1)
b = SelfUpdatingThunk(lambda: 2)

In [23]:
def f(a, b):
    print(f'adding {a} and {b}')
    return a + b

sub_expression = SelfUpdatingThunk(f, a, b)

In [24]:
value = SelfUpdatingThunk(f, sub_expression, sub_expression)

In [25]:
value()

adding 1 and 2
adding 3 and 3


6

In [26]:
value()

6

## Memoization

```python
(a + b) + (a + b)
```

In [27]:
thunk_cache = {}

def memoized_thunk(code, *args, **kwargs):
    key = (code, args, frozenset(kwargs.items()))
    try:
        thunk = thunk_cache[key]
    except KeyError:
        thunk = SelfUpdatingThunk(code, *args, **kwargs)
        thunk_cache[key] = thunk
        
    return thunk
    

In [28]:
a = memoized_thunk(lambda: 1)
b = memoized_thunk(lambda: 2)

In [29]:
def f(a, b):
    print(f'adding {a} and {b}')
    return a + b

sub_expression_1 = memoized_thunk(f, a, b)
sub_expression_2 = memoized_thunk(f, a, b)

In [30]:
value = memoized_thunk(f, sub_expression_1, sub_expression_2)

In [31]:
value()

adding 1 and 2
adding 3 and 3


6

In [32]:
value()

6

## Thank You